In [2]:
import numpy as np 
import os
import mne
import pandas as pd
from itertools import compress
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import re
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

# import the data here (Sai)


In [ ]:

#convert the data file from NIRs to CSV

#start with 50 for hbo and 30 for hbr
#find std for hbo and hbr separately
#then add std to hbo 50 and hbr 30
# preprocess the data values (worked on by Thomas and Ian)
data_path = r'C:\Users\takoy\Documents\mind-tbi-fnirs\neurotech_fnir_mind\data'
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    if not file.endswith(".snirf"):
        continue
    print(file_path)
    raw_intensity = mne.io.read_raw_snirf(file_path, verbose=True)
    raw_intensity.load_data()

    picks = mne.pick_types(raw_intensity.info, meg=False, fnirs=True)
    dists = mne.preprocessing.nirs.source_detector_distances(
        raw_intensity.info, picks = picks
    )
    raw_intensity.pick(picks[dists > 0.01])
    
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)

    raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))

    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)

    raw_haemo_unfiltered = raw_haemo.copy()
    raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)

    stim_ch = np.zeros((1, raw_haemo.n_times))

    #insert events manually here?
    sfreq = raw_haemo.info['sfreq']  # Get sampling rate
    event_interval = int(5 * sfreq)  # One event every 10 seconds
    num_events = (raw_haemo.n_times // event_interval)  # Total possible events

    manual_events = np.arange(event_interval, raw_haemo.n_times, event_interval)  # Generate event times

    for e in manual_events:
        stim_ch[0, e] = 1

    info = mne.create_info(['stim'], raw_haemo.info['sfreq'], ch_types='stim')
    stim_raw = mne.io.RawArray(stim_ch, info)
    raw_haemo.add_channels([stim_raw], force_update_info=True)

    #debugging
    print(raw_haemo.get_data().shape)

    events = mne.find_events(raw_haemo, stim_channel='stim', min_duration=0.01)
    event_dict = {"Tap": 1}

    hbo_std = np.std(raw_haemo.get_data(picks="hbo"))
    hbr_std = np.std(raw_haemo.get_data(picks="hbr"))

    reject_criteria = dict(hbo=50e-6 + hbo_std, hbr=30e-6 + hbr_std)
    tmin, tmax = -5, 15

    epochs = mne.Epochs(
        raw_haemo,
        events,
        event_id=event_dict,
        tmin=tmin,
        tmax=tmax,
        reject=reject_criteria,
        reject_by_annotation=True,
        proj=True,
        baseline=(None, 0),
        preload=True,
        detrend=None,
        verbose=True,
    )

    print("drop log here ##############################")
    epochs.plot_drop_log()

    if (len(epochs) == 0):
        print("no epochs found...")
        continue
    evoked_dict = {
        "Tapping/HbO": epochs["Tap"].average(picks="hbo"),
        "Tapping/HbR": epochs["Tap"].average(picks="hbr"),
    }

    # Rename channels until the encoding of frequency in ch_name is fixed
    for condition in evoked_dict:
        evoked_dict[condition].rename_channels(lambda x: x[:-4])

    #color_dict = dict(HbO="#AA3377", HbR="b")
    #styles_dict = dict(Control=dict(linestyle="dashed"))

    if evoked_dict:
        mne.viz.plot_compare_evokeds(
            #evoked_dict, combine="mean", ci=0.95, colors=color_dict, styles=styles_dict
            evoked_dict, combine="mean", ci=0.95,
        )
    else:
        print("No valid epochs found for plotting.")



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/takoy/fnirs-github/neurotech_fnir_mind/data'

In [9]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = None

#X_train, X_test, y_train, y_test = train_test_split(, , test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions:
y_pred = model.predict(X_test)

#Get data from fNIRs (Elan)

# calculating the mean values.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cm = metrics.confusion_matrix(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
plt.scatter(X_test, y_test, color='blue')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()


TypeError: cannot unpack non-iterable NoneType object